In [19]:
# importing libs
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, GaussianNoise
from keras.models import Model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
import random as rn
import os
from keras.models import load_model

In [20]:
# Loading data
real_input=np.loadtxt('real_input.txt')
#print(real_input.shape)
Rinput=[]
for i in range(0,32768):
    Rinput.append(real_input[i][1])
Rinput=np.array(Rinput)
Rinput=np.reshape(Rinput,(4096,8))
#print(Rinput.shape)
#print(Rinput[2])

valinput=np.loadtxt('valinput.txt')
#print(valinput.shape)
Vinput=[]
for i in range(0,8192):
    Vinput.append(real_input[i][1])
Vinput=np.array(Vinput)
Vinput=np.reshape(Vinput,(1024,8))

real_output=np.loadtxt('real_output.txt')
#print(real_output.shape)
Routput=[]
for i in range(0,32768):
    Sum=real_output[4*i][1]+real_output[4*i+1][1]+real_output[4*i+2][1]+real_output[4*i+3][1]
    mm=Sum/4
    Routput.append(mm)
Routput=np.array(Routput)
Routput=np.reshape(Routput,(4096,8))
#print(Rinput.shape)
#print(Routput[2])

valoutput=np.loadtxt('valoutput.txt')
#print(valoutput.shape)
Voutput=[]
for i in range(0,8192):
    Sum=valoutput[4*i][1]+valoutput[4*i+1][1]+valoutput[4*i+2][1]+valoutput[4*i+3][1]
    mm=Sum/4
    Voutput.append(mm)
Voutput=np.array(Voutput)
Voutput=np.reshape(Voutput,(1024,8))


In [21]:
Ir=[0,1]
If=[1,0]

Ir1=[]
If1=[]
Ir2=[]
If2=[]
for i in range(0,4096):
    Ir1.append(Ir)
    If1.append(If)
    
for i in range(0,1024):
    Ir2.append(Ir)
    If2.append(If)
    
Ir1=np.array(Ir1)
Ir2=np.array(Ir2)
If1=np.array(If1)
If2=np.array(If2)

In [22]:
#Discriminator
inputsize = 8
outputsize = 2
input_sig = Input(shape=(inputsize,))
layer1 = Dense(16, activation='relu')(input_sig)
layer2 = Dense(10, activation='relu')(layer1)
layer3 = Dense(6, activation='relu')(layer2)
p = Dense(outputsize, activation='softmax')(layer3)

discriminator = Model(input_sig, p)

discriminator.compile(optimizer='adam', loss='categorical_crossentropy')
print (discriminator.summary())

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_19 (Dense)             (None, 16)                144       
_________________________________________________________________
dense_20 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_21 (Dense)             (None, 6)                 66        
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 14        
Total params: 394
Trainable params: 394
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
#Generator
Inputsize = 8
Outputsize = 8
input_signal = Input(shape=(Inputsize,))
lay1 = Dense(30, activation='relu')(input_signal)
lay2 = Dense(20, activation='relu')(lay1)
lay3 = Dense(13, activation='relu')(lay2)
lay4 = Dense(8, activation='relu')(lay3)
lay5 = Dense(5, activation='relu')(lay4)
output_signal = Dense(Outputsize, activation='linear')(lay5)

generator = Model(input_signal, output_signal)

generator.compile(optimizer='adam', loss='categorical_crossentropy')
print (generator.summary())

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 30)                270       
_________________________________________________________________
dense_30 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_31 (Dense)             (None, 13)                273       
_________________________________________________________________
dense_32 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_33 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_34 (Dense)             (None, 8)                 48  

In [25]:
#初始化权重
keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

In [26]:
#Training od discriminator
discriminator.fit(Routput, Ir1,
                epochs=30,
                batch_size=300,
                validation_data=(Voutput, Ir2))


Train on 4096 samples, validate on 1024 samples
Epoch 1/30
4096/4096 [==============================] - 0s 22us/step - loss: 0.6632 - val_loss: 0.6239
Epoch 2/30
4096/4096 [==============================] - 0s 4us/step - loss: 0.5878 - val_loss: 0.5421
Epoch 3/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.5012 - val_loss: 0.4498
Epoch 4/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.4055 - val_loss: 0.3515
Epoch 5/30
4096/4096 [==============================] - 0s 6us/step - loss: 0.3077 - val_loss: 0.2564
Epoch 6/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.2181 - val_loss: 0.1754
Epoch 7/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.1463 - val_loss: 0.1153
Epoch 8/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.0957 - val_loss: 0.0754
Epoch 9/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.0631 - val_loss: 0.0505
Epoch 10/30
4096/4096 [==========

In [27]:
Foutput = generator.predict(Rinput) 
FVoutput=generator.predict(Vinput)

In [28]:
discriminator.fit(Foutput, If1,
                epochs=30,
                batch_size=300,
                validation_data=(FVoutput, If2))

Train on 4096 samples, validate on 1024 samples
Epoch 1/30
4096/4096 [==============================] - 0s 5us/step - loss: 5.2985 - val_loss: 2.2888
Epoch 2/30
4096/4096 [==============================] - 0s 6us/step - loss: 1.2653 - val_loss: 0.5677
Epoch 3/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.3913 - val_loss: 0.2798
Epoch 4/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.2315 - val_loss: 0.1769
Epoch 5/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.1480 - val_loss: 0.1147
Epoch 6/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.0981 - val_loss: 0.0779
Epoch 7/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.0683 - val_loss: 0.0557
Epoch 8/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.0498 - val_loss: 0.0416
Epoch 9/30
4096/4096 [==============================] - 0s 5us/step - loss: 0.0379 - val_loss: 0.0324
Epoch 10/30
4096/4096 [===========

In [29]:
generator.fit(Rinput, Routput,
                epochs=30,
                batch_size=300,
                validation_data=(Vinput, Voutput))

Train on 4096 samples, validate on 1024 samples
Epoch 1/30
4096/4096 [==============================] - 0s 34us/step - loss: 1.7880e-06 - val_loss: 3.3096e-06
Epoch 2/30
4096/4096 [==============================] - 0s 6us/step - loss: 1.9945e-06 - val_loss: 4.3918e-06
Epoch 3/30
4096/4096 [==============================] - 0s 7us/step - loss: 3.5846e-06 - val_loss: 2.1528e-06
Epoch 4/30
4096/4096 [==============================] - 0s 6us/step - loss: 3.9072e-06 - val_loss: 2.0598e-06
Epoch 5/30
4096/4096 [==============================] - 0s 6us/step - loss: 3.4553e-06 - val_loss: 3.6485e-06
Epoch 6/30
4096/4096 [==============================] - 0s 5us/step - loss: 2.6962e-06 - val_loss: -1.4452e-06
Epoch 7/30
4096/4096 [==============================] - 0s 6us/step - loss: 1.8651e-06 - val_loss: -5.1137e-07
Epoch 8/30
4096/4096 [==============================] - 0s 6us/step - loss: 2.1935e-06 - val_loss: -6.6744e-07
Epoch 9/30
4096/4096 [==============================] - 0s 5us/step 